In [1]:
import pandas as pd
#import gspread
from oauth2client.service_account import ServiceAccountCredentials
#from google.api_core.datetime_helpers import DatetimeWithNanoseconds
from datetime import datetime
from twilio.rest import Client

In [5]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials .from_json_keyfile_name('GS_creds.json', scope)
client = gspread.authorize(creds)
sheet = client.open('Lecture10').sheet1

# Reading Data

In [17]:
val = sheet.cell(1,1).value

In [18]:
val

'Hello'

In [19]:
sheet.row_values(1)

['Hello', 'World', '!']

In [22]:
sheet.get_all_values()

[['Hello', 'World', '!', ''],
 ['', 'no', '', ''],
 ['', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '1234'],
 ['', '', '', ''],
 ['', '', '', ''],
 ['', '', '', ''],
 ['dsfghstrhjtrs', '', '', '']]

In [23]:
sheet.find("dsfghstrhjtrs")

<Cell R9C1 'dsfghstrhjtrs'>

# Write Data

In [24]:
sheet.clear()

{'spreadsheetId': '1fjm8gthM2Vn2kHVNQjPfSipu13V8xLI_GQjWE1r6jMg',
 'clearedRange': 'Sheet1!A1:Z1000'}

In [26]:
sheet.update_cell(1,2, "AUTOMATION")

{'spreadsheetId': '1fjm8gthM2Vn2kHVNQjPfSipu13V8xLI_GQjWE1r6jMg',
 'updatedRange': 'Sheet1!B1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [28]:
sheet.update("A1:B2",[["v","x"],["y","z"]] )

/var/folders/2h/qlkjym5s6x91gkbp7cxjfxqh0000gn/T/ipykernel_62011/2971481360.py:1: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update("A1:B2",[["v","x"],["y","z"]] )


{'spreadsheetId': '1fjm8gthM2Vn2kHVNQjPfSipu13V8xLI_GQjWE1r6jMg',
 'updatedRange': 'Sheet1!A1:B2',
 'updatedRows': 2,
 'updatedColumns': 2,
 'updatedCells': 4}

# Adding release data to Google

In [50]:
url = "https://www.finra.org/rules-guidance/key-topics/margin-accounts/margin-statistics"
dfs = pd.read_html(url)[0]
data = dfs.iloc[0,:]

row = list(data)
row = [str(i) for i in row]
today = str(datetime.today().date())
row = [[today] + row]

sheet.append_rows(row)


{'spreadsheetId': '1fjm8gthM2Vn2kHVNQjPfSipu13V8xLI_GQjWE1r6jMg',
 'tableRange': 'Sheet1!A1:E2',
 'updates': {'spreadsheetId': '1fjm8gthM2Vn2kHVNQjPfSipu13V8xLI_GQjWE1r6jMg',
  'updatedRange': 'Sheet1!A3:E3',
  'updatedRows': 1,
  'updatedColumns': 5,
  'updatedCells': 5}}

# Text me when the release changes

In [54]:
old_num = sheet.get_all_values()[-2][2]
new_num = sheet.get_all_values()[-1][2]

In [60]:
ACCOUNT_SID = ""
AUTH_TOKEN = ""

client = Client(ACCOUNT_SID,AUTH_TOKEN)

recipients = ["+13109755351"]

message = client.messages.create(
    to = recipients,
    from_ = "+14248423165",
    body = "There has been a change in the Debit report! Old value =" + str(old_num) + "New value" + str(new_num)
)

print(f"Messge : {message.sid}")

Messge : SM4565ff99f663c37ed2aad5ef69167229


# Make as a cloud function

In [61]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.api_core.datetime_helpers import DatetimeWithNanoseconds
from datetime import datetime
from twilio.rest import Client


def send_text(old_num, new_num):
    ACCOUNT_SID = "ACfb1be2d3042ce3d3a2d4a4910e2c332f"
    AUTH_TOKEN = "11961686c414682b6a15df105751216b"
    
    client = Client(ACCOUNT_SID,AUTH_TOKEN)
    
    recipients = ["+13109755351"]
    
    message = client.messages.create(
        to = recipients,
        from_ = "+14248423165",
        body = "There has been a change in the Debit report! Old value =" + str(old_num) + "New value" + str(new_num)
    )
    
    print(f"Messge : {message.sid}")


def cloud_func(request):
    url = "https://www.finra.org/rules-guidance/key-topics/margin-accounts/margin-statistics"
    dfs = pd.read_html(url)[0]
    data = dfs.iloc[0,:]

    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials .from_json_keyfile_name('GS_creds.json', scope)
    client = gspread.authorize(creds)
    sheet = client.open('Lecture10').sheet1
    
    row = list(data)
    row = [str(i) for i in row]
    today = str(datetime.today().date())
    row = [[today] + row]
    
    sheet.append_rows(row)

    old_num = sheet.get_all_values()[-2][2]
    new_num = sheet.get_all_values()[-1][2]
    if old_num != new_num:
        send_text(old_num, new_num)
    return("YAY!")

In [2]:
import session_info

In [3]:
session_info.show()